This notebook contains some basic tests to check the model and limit setting is working correctly.

In [1]:
import numpy as np
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

from wimpy.model import Model
from wimpy.models.temp import config as base_config

In [2]:
# Create the model from scratch (takes a minute or more to sample the PDFs)
m = Model(base_config)
m.save('cs1_model')

# The next time you can just load it from disk, like so:
m = Model.load('cs1_model')

# Basic diagnostics

Energy spectra of sources included in model

In [3]:
for s in cs1_model.sources:
    s.energy_distribution.plot(label=s.label, color=s.color)
plt.yscale('log')

plt.ylabel('Events /day/kg/keV')
plt.xlabel("Energy (keV)")
plt.legend(loc='center right')
plt.ylim(1e-11, 1e-2)

NameError: name 'cs1_model' is not defined

What fraction of the total events produced an event inside the analysis space (as opposed to an event out of range or no event at all)?

In [ ]:
print([(s.name, s.fraction_in_range) for s in m.sources])

Simulate a single toy data set with "WIMP strength -1" i.e. a cross section of 10^(-46) cm^2. 

In [ ]:
truth = -1
d = m.simulate(truth)
m.show(d)
plt.legend(loc='lower right', scatterpoints=1, markerscale=2)

Get the best-fit strength, without and with considering a 3% rate uncertainty on the 50 GeV WIMP signal (the same as applied in the Bologna model for Leff, see [here](https://xecluster.lngs.infn.it/dokuwiki/doku.php?id=xenon:xenon1t:sim:notes:digangi:leff_impact_on_pl))

In [ ]:
m.bestfit(d)

In [ ]:
m.sources[-1].rate_uncertainty = 0.03
m.bestfit(d, fit_uncertainties=True)

Show the likelihood and profile likelihood curves

In [ ]:
strengths = np.linspace(-5, 2, 200)
lls = np.array([m.loglikelihood(wimp_strength=x, d=d) for x in tqdm(strengths)])
max_ll = lls.max()
plt.axvline(strengths[np.argmax(lls)], label='ML estimate', c='red', linestyle='--')
plt.plot(strengths, (max_ll-lls), label='LR')

plls = np.array([m.bestfit(guess_strength=x, d=d, fit_uncertainties=True, fit_strength=False)[1]
                 for x in tqdm(strengths)])
max_pll = plls.max()
plt.axvline(strengths[np.argmax(plls)], label='pL estimate', c='purple', linestyle='-.')
plt.plot(strengths, (max_pll-plls), label='Profile LR')

plt.xlabel('Log10(WIMP cross section (zb))')
plt.ylabel('- Log likelihood ratio')
plt.axvline(truth, label='Truth', c='r', linestyle=':')
plt.legend(loc='upper left')
plt.ylim(0, 5)
plt.xlim()
plt.show()

# p_ratio_hist(m).plot(vmin=1e-6, vmax=1e4, log_scale=True,  

#                      cblabel='s/b PDF ratio', cmap=plt.cm.Greens)
# m.show(d)
# plt.show()

Give 90% confidence intervals and 90% upper limits on the WIMP cross section:

In [ ]:
m.interval(d, kind='limit')

In [ ]:
low, high = m.interval(d, kind='central') 
print(low, high, high-low)

Same with profiling. The limit should get worse (ie. higher, ie less negative) and the interval should get broader:

In [ ]:
m.interval(d, kind='limit', profile=True)

In [ ]:
low, high = m.interval(d, kind='central', profile=True)
print(low, high, high-low)

(If I try this for a rate uncertainty on the ER background, I get only a minute effect. Actually it seems the interval gets narrowed, athough minisculely so...)

Compute the sensitivity, for comparison against the Bologna model results:

In [ ]:
def get_sensitivity(model, n_trials=1000, profile=False):
    
    bg_limits = [model.interval(model.simulate(wimp_strength=model.no_wimp_strength), 
                                kind='limit', profile=profile) 
                 for _ in tqdm(range(n_trials))]
    med = np.median(bg_limits)
    ll = np.percentile(bg_limits, stats.norm.cdf(-1) * 100)
    ul = np.percentile(bg_limits, stats.norm.cdf(1) * 100)
    print(med, (ul-ll)/2/np.sqrt(n_trials))
    #return [10**x * reference_wimp_cross_section for x in (med, ll, ul)]
    
get_sensitivity(m)
get_sensitivity(m, profile=True)

In [ ]:
# The Bologna model result expressed in the same parameter:
# From https://xecluster.lngs.infn.it/dokuwiki/doku.php?id=xenon:xenon1t:sim:notes:digangi:sensitivity-profile-likelihood-ratio#with_or_without_cls
# Sensitivit with Leff uncertainty, but without CLS is 8.6e-48
# (CLS is a factor 1.4)
np.log10(8.6*10**-48)+45